The purpose of this notebook is to calculate the power produced by a decay chain after the production of a given nuclide. Specifically, the power will be calculated for $t = 10^{n}$ seconds where $n \in \mathbb{Z}, -10 \le n \le 10$

In [1]:
using Unitful #https://painterqubits.github.io/Unitful.jl/stable/
#quantity * @u_str("unit abbreviation") 
using Symbolics #https://symbolics.juliasymbolics.org/dev/
#cite https://doi.org/10.48550/arXiv.2105.03949
using Latexify
using Test
#1 * @u_str("mA") is 1 milliamp
using CSV, DataFrames
#using Plots
using PlotlyJS
using Printf
using SymPy #https://docs.juliahub.com/SymPy/ 
using PDFIO
using Unzip
using Interpolations
using Downloads
#plotlyjs()
data_dir = ("C:\\Cross-Section-Data\\")
cross_section_dir = data_dir
parent_dir = "C:\\Users\\engin\\Documents\\GitHub\\Energy\\"

WebIO._IJuliaInit()

"C:\\Users\\engin\\Documents\\GitHub\\Energy\\"

In [3]:
decay_chain_dir = parent_dir * "ExportedData\\all_beta_decay_chains\\"
decay_chain_files = readdir(decay_chain_dir)
all_decay_chains = [CSV.read(decay_chain_dir * file, DataFrame)
                    for file in decay_chain_files]
all_decay_chains = [rename!(df, :Column1 => :Parent) for df in all_decay_chains]

3192-element Vector{DataFrame}:
 4×4 DataFrame
 Row │ Parent   e-Folding Time (seconds)  Average beta-decay energy  Daughter 
     │ String7  Float64                   Float64                    String7  
─────┼────────────────────────────────────────────────────────────────────────
   1 │ 100Ag                   173.989                         5.69  100Cd
   2 │ 100Cd                    70.8363                        5.69  100In
   3 │ 100In                     8.10795                       5.69  100Sn
   4 │ 100Sn                     1.70238                       5.69  100Sb
 3×4 DataFrame
 Row │ Parent   e-Folding Time (seconds)  Average beta-decay energy  Daughter 
     │ String7  Float64                   Float64                    String7  
─────┼────────────────────────────────────────────────────────────────────────
   1 │ 100Cd                    70.8363                        5.69  100In
   2 │ 100In                     8.10795                       5.69  100Sn
   3 │ 100Sn  

I derived the decay rate of the n-th generation nuclide in a decay chain here:

https://www.overleaf.com/project/627613def6b848465d85e5bb

https://github.com/MarcosP7635/Math-for-Energy/blob/main/main.tex

file:///C:/Users/engin/Downloads/Math_for_Energy(4).pdf 

In [4]:
factor(t, e_folding_time) = 1 - exp(t / -e_folding_time)

factor (generic function with 1 method)

In [5]:
a_prod(e_folding_times, t) = prod([factor(t, e_folding_times[k])
                                   for k in 2:length(e_folding_times)])

a_prod (generic function with 1 method)

In [6]:
r(e_fold_times, t) = (a_prod(e_fold_times, t) * exp(t / -e_fold_times[1])
                      /
                      -e_fold_times[1])


r (generic function with 1 method)

In [7]:
λ_0, λ_1, λ_2, t = @variables λ_0, λ_1, λ_2, t
expected_output = (1 - exp(t / -λ_1)) * (1 - exp(t / -λ_2)) * exp(t / -λ_0) / (-λ_0)

(-(1 - exp(t / (-λ_1)))*(1 - exp(t / (-λ_2)))*exp(t / (-λ_0))) / λ_0

In [8]:
e_folding_times = [λ_0, λ_1, λ_2]
r(e_folding_times, t)
Test.@test r(e_folding_times, t) - expected_output == 0

Test Passed
  Expression: r(e_folding_times, t) - expected_output == 0
   Evaluated: 0 == 0

It works! :) 

The average beta decay energy is given in keV. 

In [9]:
joules_per_keV = Rational(uconvert(@u_str("J"), 1 * @u_str("keV")) / @u_str("J"))

337//2103388558093277156

Avogadro's number

In [10]:
N_A = 6.02214076e23
conversion_factor = Rational(joules_per_keV * N_A)

6475021031458043//67108864

Now we calculate the power in watts

In [11]:
power_per_mole(decay_rate, decay_energy) = decay_rate * decay_energy * conversion_factor

power_per_mole (generic function with 1 method)

In [12]:
power_per_mole(e_folding_times, decay_energies, t) = sum([
    power_per_mole(r(e_folding_times[1:i], t), decay_energies[i])
    for i in 1:length(decay_energies)])

power_per_mole (generic function with 2 methods)

In [13]:
E_0, E_1, E_2, t = @variables E_0, E_1, E_2, t
expected_output = conversion_factor * sum([
    E_0 * exp(t / -λ_0) / (-λ_0),
    (1 - exp(t / -λ_1)) * E_1 * exp(t / -λ_0) / (-λ_0),
    (1 - exp(t / -λ_1)) * (1 - exp(t / -λ_2)) * E_2 * exp(t / -λ_0) / (-λ_0)
])

((-6475021031458043//67108864)*E_0*exp(t / (-λ_0))) / λ_0 + ((-6475021031458043//67108864)*E_1*(1 - exp(t / (-λ_1)))*exp(t / (-λ_0))) / λ_0 + ((-6475021031458043//67108864)*E_2*(1 - exp(t / (-λ_1)))*(1 - exp(t / (-λ_2)))*exp(t / (-λ_0))) / λ_0

In [14]:
decay_energies = [E_0, E_1, E_2]
Test.@test Symbolics.simplify(power_per_mole(e_folding_times, decay_energies, t) 
            - expected_output) == 0

Test Passed
  Expression: Symbolics.simplify(power_per_mole(e_folding_times, decay_energies, t) - expected_output) == 0
   Evaluated: 0 == 0

In [16]:
x = [power_per_mole(df[!,"e-Folding Time (seconds)"], df[!,"Average beta-decay energy"], 1)
for df in all_decay_chains];